In [1]:
from pyspark import SparkConf, SparkContext

# Stop the existing SparkContext
SparkContext.getOrCreate().stop()

# Create a Spark configuration object
conf = SparkConf()

# Set the default parallelism
# conf.set("spark.default.parallelism", "200")
conf.set("spark.sql.shuffle.partitions", "20")

24/05/30 14:23:33 WARN Utils: Your hostname, Ugurs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.30 instead (on interface en0)
24/05/30 14:23:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/30 14:23:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pySparkSetup').config(conf=conf).getOrCreate()

In [4]:
spark.conf.set('spark.sql.adaptive.enabled', 'false')

In [5]:
parquet_path= "/Users/ugurkalkavan/Downloads/m06sparkbasics/weather"

In [6]:
spark.sparkContext.setJobDescription('read parquet first')  # Setting Job description

In [7]:
weather = spark.read.parquet(parquet_path)

In [8]:
from pyspark.sql.functions import col,approx_count_distinct

In [9]:
spark.sparkContext.setJobDescription('approx_count_distinct')  # Setting Job description

In [18]:
weather.agg(approx_count_distinct(col("lng"))).show()

24/05/30 14:23:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------------+
|approx_count_distinct(lng)|
+--------------------------+
|                    324657|
+--------------------------+



In [19]:
spark.sparkContext.setJobDescription('count')  # Setting Job description

In [20]:
weather.select(col("lng")).count()

112394743

In [21]:
spark.sparkContext.setJobDescription('distinct')  # Setting Job description

In [22]:
weather.select(col("lng")).distinct().count()

339474

In [36]:
spark.sparkContext.setJobDescription('count all')  # Setting Job description

In [38]:
weather.count()

112394743

In [24]:
spark

In [25]:
weather.rdd.getNumPartitions()

16

In [ ]:
weather.

In [10]:
weather = weather.repartition(16)

In [11]:
weather.rdd.getNumPartitions()

16

In [13]:
spark.sparkContext.setJobDescription('Count the year month day')  # Setting Job description

In [14]:
weather = weather.groupby("year","month","day").count()

In [15]:
weather.show()

+----+-----+---+-------+
|year|month|day|  count|
+----+-----+---+-------+
|2017|    9|  7|1230518|
|2017|    9| 30|1230518|
|2017|    9| 15|1230518|
|2017|    8|  4|1230518|
|2017|    9| 25|1230518|
|2017|    8|  6|1230518|
|2016|   10| 21|1204295|
|2016|   10| 14|1204295|
|2016|   10|  8|1204295|
|2017|    8| 29|1230518|
|2017|    9| 21|1230518|
|2017|    8| 12|1230518|
|2016|   10|  1|1204295|
|2017|    9|  9|1230518|
|2017|    9|  6|1230518|
|2017|    8| 19|1230518|
|2016|   10| 12|1204295|
|2016|   10| 25|1204295|
|2016|   10|  4|1204295|
|2016|   10| 22|1204295|
+----+-----+---+-------+
only showing top 20 rows



In [16]:
weather.rdd.getNumPartitions()

20

In [17]:
spark.sparkContext.setJobDescription('do noop')  # Setting Job description

In [18]:

spark.sql("SET spark.sql.shuffle.partitions=10")


#spark.read.parquet(parquet_path).write.format("noop").mode("overwrite").save() ## Test with a noop write

weather.write.format("noop").mode("overwrite").save() ## Test with a noop write

In [19]:
spark.sparkContext.defaultParallelism

200

In [20]:
spark

In [21]:
weather.rdd.getNumPartitions()

20

In [22]:
spark.conf.get("spark.sql.shuffle.partitions")

'10'

In [34]:
from pyspark import SparkConf, SparkContext

sc = spark.sparkContext
app_id = sc.applicationId
print(app_id)

local-1716992376481


In [16]:
import requests
import statistics
from datetime import datetime

def identify_parallel_stages(app_id):   
    base_url = "http://localhost:4040/api/v1/applications/" # change to your Spark master node hostname
    stages_resp =  requests.get(f"{base_url}{app_id}/stages/")
    stages = stages_resp.json()

    for stage in stages:
        if 'submissionTime' in stage and stage['submissionTime'] is not None:
            stage['submissionTime'] = datetime.strptime(stage['submissionTime'], '%Y-%m-%dT%H:%M:%S.%fGMT')

    stages.sort(key= lambda x: x['submissionTime'] if 'submissionTime' in x and x['submissionTime'] is not None else datetime.min)
    
    # Identify and print parallel stage info
    parallel_groups = []
    curr_group = [stages[0]]
    for i in range(1, len(stages)):
        if stages[i]['submissionTime'] < curr_group[-1]['completionTime']:
            curr_group.append(stages[i])
        else:
            parallel_groups.append(curr_group)
            curr_group = [stages[i]]
    parallel_groups.append(curr_group)

    for i, group in enumerate(parallel_groups):
        print(f"\nParallel Group {i+1}: {len(group)} stages ran in parallel")
        for stage in group:
            print(f"Stage ID: {stage['stageId']}, Name: {stage['name']}, Submission Time: {stage['submissionTime']}, Completion Time: {stage['completionTime']}")

            

In [18]:
def print_stage_info(app_id):  
    base_url = "http://localhost:4040/api/v1/applications/" # change to your Spark master node hostname
    stages_resp =  requests.get(f"{base_url}{app_id}/stages/")
    stages = stages_resp.json()

    # Analyze tasks and skew for each stage
    for stage in stages:
        response_tasks = requests.get(f"{base_url}{app_id}/stages/{stage['stageId']}/{stage['attemptId']}/taskList")
        tasks = response_tasks.json()

        if tasks:
            tasks_durations = [task['taskMetrics']['executorRunTime'] for task in tasks]
            min_task_duration = min(tasks_durations)
            median_task_duration = statistics.median(tasks_durations)
            max_task_duration = max(tasks_durations)
            task_skew = max_task_duration/median_task_duration if median_task_duration > 0 else 0
            task_skew = round(task_skew, 2)
        
            # Collect additional spill and shuffle metrics
            memory_spill = sum(task['taskMetrics']['memoryBytesSpilled'] for task in tasks)
            disk_spill = sum(task['taskMetrics']['diskBytesSpilled'] for task in tasks)
            shuffle_read = sum(task['taskMetrics']['shuffleReadMetrics']['totalBytesRead'] for task in tasks if 'shuffleReadMetrics' in task['taskMetrics'])
            shuffle_write = sum(task['taskMetrics']['shuffleWriteMetrics']['shuffleBytesWritten'] for task in tasks if 'shuffleWriteMetrics' in task['taskMetrics'])

            print(f"\nAnalysis for Stage ID: {stage['stageId']}, Stage name: {stage['name']}")
            print(f"Number of tasks: {stage['numTasks']}")
            print(f"Number of active tasks: {stage['numActiveTasks']}")
            print(f"Number of completed tasks: {stage['numCompleteTasks']}")
            print(f"Median task duration: {median_task_duration} ms")
            print(f"Max task duration: {max_task_duration} ms")
            print(f"Min task duration: {min_task_duration} ms")
            print(f"Task Skew: {task_skew}")  
            print(f"Memory spill: {memory_spill} bytes")
            print(f"Disk spill: {disk_spill} bytes")
            print(f"Shuffle read: {shuffle_read} bytes")
            print(f"Shuffle write: {shuffle_write} bytes") 



In [20]:

print_stage_info(app_id)

KeyError: 'totalBytesRead'

In [210]:
identify_parallel_stages(app_id)

KeyError: 'submissionTime'

In [22]:
base_url = "http://localhost:4040/api/v1/applications/" # change to your Spark master node hostname
stages_resp =  requests.get(f"{base_url}{app_id}/stages/")
stages = stages_resp.json()

In [26]:
stages[1]

{'status': 'COMPLETE',
 'stageId': 0,
 'attemptId': 0,
 'numTasks': 1,
 'numActiveTasks': 0,
 'numCompleteTasks': 1,
 'numFailedTasks': 0,
 'numKilledTasks': 0,
 'numCompletedIndices': 1,
 'submissionTime': '2024-05-28T20:50:25.690GMT',
 'firstTaskLaunchedTime': '2024-05-28T20:50:26.304GMT',
 'completionTime': '2024-05-28T20:50:27.172GMT',
 'executorDeserializeTime': 515,
 'executorDeserializeCpuTime': 449006000,
 'executorRunTime': 264,
 'executorCpuTime': 96017000,
 'resultSize': 1869,
 'jvmGcTime': 0,
 'resultSerializationTime': 10,
 'memoryBytesSpilled': 0,
 'diskBytesSpilled': 0,
 'peakExecutionMemory': 0,
 'inputBytes': 0,
 'inputRecords': 0,
 'outputBytes': 0,
 'outputRecords': 0,
 'shuffleRemoteBlocksFetched': 0,
 'shuffleLocalBlocksFetched': 0,
 'shuffleFetchWaitTime': 0,
 'shuffleRemoteBytesRead': 0,
 'shuffleRemoteBytesReadToDisk': 0,
 'shuffleLocalBytesRead': 0,
 'shuffleReadBytes': 0,
 'shuffleReadRecords': 0,
 'shuffleCorruptMergedBlockChunks': 0,
 'shuffleMergedFetchFall

In [ ]:
    # Analyze tasks and skew for each stage
    for stage in stages:
        response_tasks = requests.get(f"{base_url}{app_id}/stages/{stage['stageId']}/{stage['attemptId']}/taskList")
        tasks = response_tasks.json()
        for task in tasks:
            print(task)
            print()

In [ ]:
tasks

In [21]:
spark

In [63]:
sc.defaultParallelism

10

In [130]:
import requests
import json

base_url = "http://localhost:4040/api/v1/applications/"
stages_resp =  requests.get(f"{base_url}{app_id}/stages/")
stages = stages_resp.json()

tasks_data = []  # Initialize the JSON list
# Analyze tasks and skew for each stage
for stage in stages:
    response_tasks = requests.get(f"{base_url}{app_id}/stages/{stage['stageId']}/{stage['attemptId']}/taskList")
    tasks = response_tasks.json()
    tasks_data.append(tasks)  # Add tasks data in list

# Convert tasks_data list to JSON 
json_data = json.dumps(tasks_data)

In [131]:
df = spark.read.json(spark.sparkContext.parallelize([json_data]))  # Use the json method to read the json_data

In [134]:
df.show()

+------------------+-------+--------+----------+-----------------+----+-----+----------+-----------+--------------+-----------+------+------+------------+-----------+
|accumulatorUpdates|attempt|duration|executorId|gettingResultTime|host|index|launchTime|partitionId|schedulerDelay|speculative|status|taskId|taskLocality|taskMetrics|
+------------------+-------+--------+----------+-----------------+----+-----+----------+-----------+--------------+-----------+------+------+------------+-----------+
|              NULL|   NULL|    NULL|      NULL|             NULL|NULL| NULL|      NULL|       NULL|          NULL|       NULL|  NULL|  NULL|        NULL|       NULL|
+------------------+-------+--------+----------+-----------------+----+-----+----------+-----------+--------------+-----------+------+------+------------+-----------+



In [111]:
df2 = df.filter("Event='SparkListenerTaskEnd'").select("Stage ID", "Task Info.*", "Task Metrics.*")

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Event` cannot be resolved. Did you mean one of the following? [`host`, `index`, `attempt`, `status`, `taskId`].; line 1 pos 0;
'Filter ('Event = SparkListenerTaskEnd)
+- LogicalRDD [accumulatorUpdates#186, attempt#187L, duration#188L, executorId#189, gettingResultTime#190L, host#191, index#192L, launchTime#193, partitionId#194L, schedulerDelay#195L, speculative#196, status#197, taskId#198L, taskLocality#199, taskMetrics#200], false
